In [1]:
import os
import pandas as pd
import random
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pyra.dst import handle_dst_dates
from pyra.loadagg import average_previous_and_subsequent_dates, average_previous_and_subsequent_hours
from pyra.date_utils import daily_index, hourly_index, planning_year
import shutil
import sys
from getpass import getpass
import win32com.client
import pythoncom

# Scraping libraries
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from urllib.parse import urljoin
import time
import zipfile

## EMTDB Connection

In [ ]:
sys.path.append(r'K:\Valuation\_Analysts\Hemanth\Python Notebooks\Miscellaneous\Python Analyst Engine 2.0')

In [ ]:
from util import EmtdbConnection
from emtdb_api import pull_lmp_data

In [ ]:
user = 'HXH07BP'
pw = getpass('Enter EMTDB pass:')

In [ ]:
emtdb = EmtdbConnection(user, pw)

## Data Scraping

In [ ]:
# Set up session (some sites require headers)
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
})

# Get the page
url = 'https://bgs-auction.com/bgs.dataroom.asp' # monthly data page
# url = 'https://bgs-auction.com/bgs.dataroom.occ.asp' # additional data page
# url = 'https://www.bgs-auction.com/bgs.auction.regproc.asp#sec2' # Documents page
response = session.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links that point to files (adjust selectors as needed)
file_links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    # Look for common file extensions
    extensions = [
        # '.pdf',
        # '.xlsx',
        # '.xls',
        '.xlsb',
        # '.csv',
        # '.zip',
        # '.doc',
        # '.docx'
    ]
    if any(href.lower().endswith(ext) for ext in extensions):
        full_url = urljoin(url, href)
        file_links.append(full_url)

# Create download directory
download_dir = Path('bgs_files')
download_dir.mkdir(exist_ok=True)

# Download each file
for file_url in file_links:
    filename = Path(file_url).name
    print(f"Downloading {filename}...")
    
    try:
        file_response = session.get(file_url, timeout=30)
        file_response.raise_for_status()
        
        filepath = download_dir / filename
        with open(filepath, 'wb') as f:
            f.write(file_response.content)
        print(f"✓ Saved {filename}")
        
        time.sleep(0.5)  # Be polite to the server
    except Exception as e:
        print(f"✗ Failed to download {filename}: {e}")

print(f"\nDownloaded {len(file_links)} files to {download_dir}")

Unzipping files

In [ ]:
# Specify the folder containing zip files
zip_folder = Path('bgs_files')  # or wherever your zips are

# Find and extract all zip files
for zip_path in zip_folder.glob('*.zip'):
    print(f"Extracting {zip_path.name}...")
    
    # Create extraction folder (same name as zip file)
    extract_to = zip_folder / zip_path.stem
    extract_to.mkdir(exist_ok=True)
    
    # Extract
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    print(f"✓ Extracted to {extract_to}")

## Data Aggregation and Analysis

### Setting up file paths and helper functions

Changes made to aid aggregation:
1. Rename the string 'PSE&G' in newer file names to 'PSEG' to match old file naming convention
2. Replace '_' in file_name by '-' to match old file naming convention
3. Replace Feburary with February in Hist_CIEP_Switching_Statistics.xlsx

In [ ]:
# Main folder paths

zones = ['ACE', 'JCP&L', 'JCP&L_Excess_Generation', 'PSEG', 'RECO']
zones_without_gen = ['ACE', 'JCP&L', 'PSEG', 'RECO']

# The below folder has information that is updated less frequently
less_frequent_data_folder_path = r"K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Data\bgs_files_less_frequent"

# The below folder has information that is updated on a monthly basis
monthly_data_folder_path = r"K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Data\bgs_files_monthly" 

# Historical data folder names

ace_historical_folder_name = 'ACE_Historical_Data_(Prior_to_June_2022)'
jcpl_historical_folder_name = 'JCP&L_Historical_Data_(Prior_to_June_2022)'
pseg_historical_folder_name = 'PSE&G_Historical_Data_(Prior_to_June_2022)'
reco_historical_folder_name = 'RECO_Historical_Data_(Prior_to_June_2022)'
recent_folder_name = 'CIEP_Recent_Data_(After_June_2022)'
switching_folder_name = 'Switching_Statistics_(Prior_to_June_2022)'

folder_names = [
    ace_historical_folder_name,
    jcpl_historical_folder_name,
    pseg_historical_folder_name,
    reco_historical_folder_name,
    recent_folder_name
]

folder_names_without_recent = folder_names[:-1]

In [ ]:
# Helper functions

def flatten_columns(DF, sep='_'):
    temp_DF = DF.copy()
    temp_DF.columns = [sep.join(map(str, col)).strip(sep) for col in temp_DF.columns]
    return temp_DF

### Load

#### Helper functions to aggregate load

In [ ]:
# Function to aggregate load

def aggregate_load(folder_path, file_name, file_type):
    # Reads load from given file_path and returns DataFrame. We have slightly different code for csv and excel since the file names and formats are slightly different.

    print(f'Aggregating load data from {file_name}')

    if file_type == 'csv':
        
        if 'Generation' not in file_name:

            if 'Eligible' not in file_name:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[2].split('-')[0]
            else:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[2].split('-')[1]

        else: # If the file_name contains the word 'Generation' - for JCP&L

            if 'Eligible' not in file_name:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[5] + '_' + file_name.split('_')[6][:-4] + '_' + file_name.split('_')[2] # To remove the .csv string at the end
            else:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[5] + '_' + file_name.split('_')[6][:-4]  + '_' + file_name.split('_')[3] # To remove the .csv string at the end

        return pd.read_csv(
                    os.path.join(folder_path, file_name),
                    skiprows=4
                    ).melt(
                    id_vars='Unnamed: 0'
                ).rename(
                    columns={
                        'Unnamed: 0': 'Date',
                        'variable': 'Hour',
                        'value': 'Load',
                    }
                ).assign(
                    Hour=lambda DF: DF.Hour.str[-2:].astype(int),
                    Date=lambda DF: pd.to_datetime(DF.Date),
                    Load_Name=Load_Name
                ).sort_values(
                    by=['Date', 'Hour']
                )
    
    elif file_type == 'excel':
        
        if 'Generation' not in file_name:

            if 'Eligible' not in file_name:
                Load_Name = file_name.split('_')[0] + '_' + file_name.split('_')[1].split('-')[0]
            else:
                Load_Name = file_name.split('_')[0] + '_' + file_name.split('_')[1].split('-')[1]

        else: # If the file_name contains the word 'Generation' - for JCP&L

            if 'Eligible' not in file_name:
                Load_Name = file_name.split('_')[0] + '_' + file_name.split('_')[4] + '_' + file_name.split('_')[5] + '_' + file_name.split('_')[1]
            else:
                Load_Name = file_name.split('_')[0] + '_' + file_name.split('_')[3] + '_' + file_name.split('_')[4] + '_' + file_name.split('_')[1].split('-')[1]    

        if 'xlsx' in file_name: # For usual excel files
            engine=None
        elif 'xlsb' in file_name:
            engine='pyxlsb'

        return pd.read_excel(
                    os.path.join(folder_path, file_name),
                    engine=engine,
                    skiprows=5 # The format of the excel files are shifted by a row compared to the csv files
                    ).melt(
                    id_vars='Unnamed: 0'
                ).rename(
                    columns={
                        'Unnamed: 0': 'Date',
                        'variable': 'Hour',
                        'value': 'Load',
                    }
                ).assign(
                    Hour=lambda DF: DF.Hour.str[-2:].astype(int),
                    # Date=lambda DF: pd.to_datetime(DF.Date),
                    Date=lambda DF: pd.to_datetime('1899-12-30') + pd.to_timedelta(DF.Date, unit='D') if 'xlsb' in file_name else pd.to_datetime(DF.Date), # Adding this since pandas messes up reading dates from xlsb files sometimes
                    Load_Name=Load_Name
                ).sort_values(
                    by=['Date', 'Hour']
                )             

In [ ]:
df_load = pd.DataFrame()

for folder_name in folder_names:
    
    folder_path = os.path.join(monthly_data_folder_path, folder_name, 'CIEP') # Only aggregating CIEP data
    
    for file_name in os.listdir(folder_path):

        if ('Load' in file_name) or ('Generation' in file_name): # Adding this filter only to aggregate load data first. 'Generation' is also included for JCP&L
            
            if 'Recent' in folder_name:
                df_load_temp = aggregate_load(folder_path, file_name, 'excel') # Newer files in excel
            else:
                df_load_temp = aggregate_load(folder_path, file_name, 'csv') # Older files in csv
        
            df_load = pd.concat([df_load, df_load_temp])

df_load

#### Doing some checks before aggregating load

Observed a number of NaNs for Hour 2, 3 and 25 and some ' -   ' for the following 2 data points

In [ ]:
# Hours with missing load

df_load[df_load.Load.isna()].Hour.unique()

In [ ]:
# Rows with non-numeric load

df_load.dropna()[pd.to_numeric(df_load.dropna()['Load'], errors='coerce').isna()]

#### Pivoting load

In [ ]:
df_load_pivoted = df_load.dropna().replace( # Dropping NaNs which correspond to hours 2, 3 and 25 and replacing the weird string with NaN
    ' -   ', np.nan 
).assign(
    Load=lambda DF: DF.Load.astype(float)
).pipe(
    lambda DF: DF[DF.Hour != 25] # Getting rid of Hour 25
).pivot(
    index=['Date', 'Hour'],
    columns='Load_Name',
    values='Load'
).sort_index(
    level=[
        'Date', 'Hour'
    ]
).reset_index().pipe(
    lambda DF: DF[DF.Date >= '2014-01-01'] # Filtering out data before 2014. Raw data starts on 2001-08-01.
).set_index(
    ['Date', 'Hour']
)


#### Load adjustments - very tedious

In [ ]:
# Doing the following adjustments before DST adjustments since the hour 2 (and sometimes 4) seems to have been doubled on these dates right before the DST 

for date in ['2016-03-13', '2020-03-08', '2024-03-10']:
    df_load_pivoted['PSEG_BGS'].loc[
    pd.IndexSlice[date, 2]
] = df_load_pivoted['PSEG_BGS'].loc[
    pd.IndexSlice[date, 2]
] / 2

    df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice[date, 2]
] = df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice[date, 2]
] / 2


df_load_pivoted['PSEG_BGS'].loc[
    pd.IndexSlice['2022-03-13', 4]
] = df_load_pivoted['PSEG_BGS'].loc[
    pd.IndexSlice['2022-03-13', 4]
] / 2

df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2022-03-13', 4]
] = df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2022-03-13', 4]
] / 2

In [ ]:
# Replacing existing values with NaN so that they can be interpolated. Note that only the specific range is intepolated since the other NaNs will be handled separately by 
# the DST function

df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2020-09-19', 10:16]
] = np.nan

df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2020-09-20', 9:17]
] = np.nan

df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2020-09-19', 1:24]
] = df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2020-09-19', 1:24]
].interpolate()

df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2020-09-20', 1:24]
] = df_load_pivoted['PSEG_Eligible'].loc[
    pd.IndexSlice['2020-09-20', 1:24]
].interpolate()

Doing the DST adjustments zone by zone since the function handle_dst_dates cannot handle all of them at once if the missing hours are not the same across all the loads

In [ ]:
for zone in zones:
    print(f'Doing DST adjustments for {zone} load \n')
    df_load_pivoted[[zone + '_BGS', zone + '_Eligible']] = df_load_pivoted[[zone + '_BGS', zone + '_Eligible']].pipe(
        handle_dst_dates,
        verbose=True
    )
    print('\n')

Outliers found in data:

1. ACE_BGS and ACE_Eligible has some outlier data points on 2016-04-28, 2017-10-18 and 2024-05-11 to 2024-05-13
2. RECO_BGS has some drops from 2019-04-24 through 2019-04-30 and 2024-01-31
3. RECO_Eligible have some drops from 2019-04-24 through 2019-04-30
4. PSEG_Eligible has some high spikes on 2 dates which have been averaged by the previous and next hours
5. RECO_BGS and RECO_Eligible have the DST ending hours doubled on certain years

In [ ]:
# Original approach
# df_load_pivoted['ACE_BGS'] = average_previous_and_subsequent_dates(
#     df_load_pivoted['ACE_BGS'],
#     date=pd.to_datetime('2016-04-28')
# )

# Holding flat to the previous day for ACE_BGS and ACE_Eligible on 2016-04-28

df_load_pivoted['ACE_BGS'].loc[
    pd.IndexSlice['2016-04-28', 1:24]
] = df_load_pivoted['ACE_BGS'].loc[
    pd.IndexSlice['2016-04-27', 1:24]
].to_numpy()

df_load_pivoted['ACE_Eligible'].loc[
    pd.IndexSlice['2016-04-28', 1:24]
] = df_load_pivoted['ACE_Eligible'].loc[
    pd.IndexSlice['2016-04-27', 1:24]
].to_numpy()



# Original approach
# df_load_pivoted['ACE_BGS'] = average_previous_and_subsequent_dates(
#     df_load_pivoted['ACE_BGS'],
#     date=pd.to_datetime('2017-10-18')
# )

# Holding flat to the previous day for ACE_BGS and ACE_Eligible on 2017-10-18

df_load_pivoted['ACE_BGS'].loc[
    pd.IndexSlice['2017-10-18', 1:24]
] = df_load_pivoted['ACE_BGS'].loc[
    pd.IndexSlice['2017-10-17', 1:24]
].to_numpy()

df_load_pivoted['ACE_Eligible'].loc[
    pd.IndexSlice['2017-10-18', 1:24]
] = df_load_pivoted['ACE_Eligible'].loc[
    pd.IndexSlice['2017-10-17', 1:24]
].to_numpy()


# Holding flat to the previous week for ACE_eligible between 2024-05-11 and 2024-05-13
for date in pd.date_range('2024-05-11', '2024-05-13'):
    df_load_pivoted['ACE_Eligible'].loc[
    pd.IndexSlice[date, 1:24]
] = df_load_pivoted['ACE_Eligible'].loc[
    pd.IndexSlice[date - pd.DateOffset(7), 1:24]
].to_numpy()



# Original approach
# df_load_pivoted['RECO_BGS'] = average_previous_and_subsequent_dates(
#     df_load_pivoted['RECO_BGS'],
#     date=pd.to_datetime('2024-01-31')
# )

# Replacing the data in hours 13 through 24 on 2019-04-24 with those from 2019-04-23 for RECO_BGS

df_load_pivoted['RECO_BGS'].loc[
    pd.IndexSlice['2019-04-24', 13:24]
] = df_load_pivoted['RECO_BGS'].loc[
    pd.IndexSlice['2019-04-23', 13:24]
].to_numpy()


# Replacing the data in hours 20 through 24 on 2024-04-31 with those from 2024-04-30 for RECO_BGS

df_load_pivoted['RECO_BGS'].loc[
    pd.IndexSlice['2024-01-31', 20:24]
] = df_load_pivoted['RECO_BGS'].loc[
    pd.IndexSlice['2024-01-30', 20:24]
].to_numpy()


# Replacing the data from 2019-04-25 through 2019-04-30 with the data from one week back for RECO_BGS
for date in pd.date_range('2019-04-25', '2019-04-30'):
    df_load_pivoted['RECO_BGS'].loc[
        pd.IndexSlice[date, :]
    ] = df_load_pivoted['RECO_BGS'].loc[
    pd.IndexSlice[date - pd.DateOffset(7), :]
].to_numpy()


# Original approach
# Replacing the data from 2019-04-25 through 2019-04-30 with the data on 2019-04-23 for RECO_BGS
# for date in pd.date_range('2019-04-25', '2019-04-30'):
#     df_load_pivoted['RECO_BGS'].loc[
#         pd.IndexSlice[date, :]
#     ] = df_load_pivoted['RECO_BGS'].loc[
#     pd.IndexSlice['2019-04-23', :]
# ].to_numpy()
    

# Replacing the data in hours 13 through 24 on 2019-04-24 with those from 2019-04-23 for RECO_Eligible


df_load_pivoted['RECO_Eligible'].loc[
    pd.IndexSlice['2019-04-24', 13:24]
] = df_load_pivoted['RECO_Eligible'].loc[
    pd.IndexSlice['2019-04-23', 13:24]
].to_numpy()


# Replacing the data from 2019-04-25 through 2019-04-30 with the data from one week back for RECO_Eligible
for date in pd.date_range('2019-04-25', '2019-04-30'):
    df_load_pivoted['RECO_Eligible'].loc[
        pd.IndexSlice[date, :]
    ] = df_load_pivoted['RECO_Eligible'].loc[
    pd.IndexSlice[date - pd.DateOffset(7), :]
].to_numpy()


# Original approach

# Replacing the data from 2019-04-25 through 2019-04-30 with the data on 2019-04-23 for RECO_Eligible

# for date in pd.date_range('2019-04-25', '2019-04-30'):
#     df_load_pivoted['RECO_Eligible'].loc[
#         pd.IndexSlice[date, :]
#     ] = df_load_pivoted['RECO_Eligible'].loc[
#     pd.IndexSlice['2019-04-23', :]
# ].to_numpy()

In [ ]:
# Halving the hour 2 for specific DST dates for RECO to match secondary - so stupid how much we care so much about "tying" on our desk

reco_dst_halve_he_2_dates = [
    '2015-11-01',
    '2016-11-06',
    '2017-11-05',
    '2018-11-04',
    '2019-11-03',
    '2020-11-01',
    '2021-11-07',
    '2023-11-05'
]

for date in reco_dst_halve_he_2_dates:
    df_load_pivoted['RECO_BGS'].loc[
        pd.IndexSlice[date, 2]
    ] = df_load_pivoted['RECO_BGS'].loc[
    pd.IndexSlice[date, 2]
] / 2
    
    df_load_pivoted['RECO_Eligible'].loc[
        pd.IndexSlice[date, 2]
    ] = df_load_pivoted['RECO_Eligible'].loc[
    pd.IndexSlice[date, 2]
] / 2

In [ ]:
# Fixing the outliers in the PSEG Eligible data

# Original approach
# df_load_pivoted[['PSEG_Eligible']] = average_previous_and_subsequent_hours(
#     df_load_pivoted[['PSEG_Eligible']],
#     pd.to_datetime('2016-03-13'),
#     2
# )

# Original approach
# df_load_pivoted[['PSEG_Eligible']] = average_previous_and_subsequent_hours(
#     df_load_pivoted[['PSEG_Eligible']],
#     pd.to_datetime('2020-03-08'),
#     2
# )

# df_load_pivoted[['PSEG_Eligible']] = average_previous_and_subsequent_hours(
#     df_load_pivoted[['PSEG_Eligible']],
#     pd.to_datetime('2022-03-13'),
#     4
# )

df_load_pivoted[['PSEG_Eligible']] = average_previous_and_subsequent_hours(
    df_load_pivoted[['PSEG_Eligible']],
    pd.to_datetime('2024-03-10'),
    2
)

#### Calculating JCP&L Excess Gen Load and plotting all the loads

Since excess generation is no longer part of the pricing load for JCP&L, we don't want to distort history by adding it. So even though I add it to below, we end up not using excess gen. 

In [ ]:
df_load_pivoted = df_load_pivoted.assign(
    JCPL_Net_BGS=lambda DF: (DF['JCP&L_BGS'] + DF['JCP&L_Excess_Generation_BGS']).where(DF.index.get_level_values(0) < '2022-08-01', DF['JCP&L_BGS']),
    JCPL_Net_Eligible=lambda DF: (DF['JCP&L_Eligible'] + DF['JCP&L_Excess_Generation_Eligible']).where(DF.index.get_level_values(0) < '2022-08-01', DF['JCP&L_Eligible'])
).rename(
    columns={
        'JCPL_Net_BGS': 'JCP&L_Net_BGS',
        'JCPL_Net_Eligible': 'JCP&L_Net_Eligible'
    }
).reindex(
    columns=[zone + '_BGS' for zone in zones] + ['JCP&L_Net_BGS'] + [zone + '_Eligible' for zone in zones] + ['JCP&L_Net_Eligible'] # following format requested by secondary 
)

df_load_pivoted

In [ ]:
for zone in zones:
    fig = df_load_pivoted.reset_index().pipe(
        px.line,
        x='Date',
        # y=f'{zone}_BGS',
        y=f'{zone}_BGS',
        # y=[zone + '_BGS' for zone in zones],
        # y=[zone + '_Eligible' for zone in zones],
        title=f'{zone}_BGS',
        labels={
            'y': 'Load'
        }
    )

    fig.show()


In [ ]:
df_load_pivoted

### Raw Tags

In [ ]:
# Function to aggregate tags

def aggregate_tags(folder_path, file_name, file_type):
    # Reads tags from given file_path and returns DataFrame. We have slightly different code for csv and excel since the file names and formats are slightly different.

    if file_type == 'csv':
        
        if 'Generation' not in file_name:

            if 'Eligible' not in file_name:
                if 'Retail' not in file_name: # Adding this additional check since we're also aggregating RSCP and TOTAL tags for verification purposes
                    Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[2].split('-')[0]
                else:
                    Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[3]
            else:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[2].split('-')[1]

        else: # If the file_name contains the word 'Generation' - for JCP&L
            
            if 'Eligible' not in file_name:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[2] + '_' + file_name.split('_')[5] + '_' + file_name.split('_')[6][:-4] # To remove the .csv string at the end
            else:
                Load_Name = file_name.split('_')[1] + '_' + file_name.split('_')[3] + '_' + file_name.split('_')[5] + '_' + file_name.split('_')[6][:-4] # To remove the .csv string at the end

        print(f'Aggregating tag data from {file_name} with load name {Load_Name}')

        return pd.read_csv(
            os.path.join(folder_path, file_name),
            skiprows=4
        ).rename(
            columns={
                'Unnamed: 0': 'Date'
            }
        ).rename(
            columns=lambda x: x.strip() # Adding this since some files have spaces in the column names
        ).assign(
            Load_Name=Load_Name
        )
    
    elif file_type == 'excel':
        
        if 'Eligible' not in file_name:
            Load_Name = file_name.split('_')[0] + '_' + file_name.split('_')[1].split('-')[0]
        else:
            Load_Name = file_name.split('_')[0] + '_' + file_name.split('_')[1].split('-')[1]        

        print(f'Aggregating tag data from {file_name} with load name {Load_Name}')

        return pd.read_excel(
            os.path.join(folder_path, file_name),
            engine='pyxlsb',
            skiprows=5
        ).rename(
            columns={
                'Unnamed: 0': 'Date'
            }
        ).rename(
            columns=lambda x: x.strip() # Adding this since some files have spaces in the column names
        ).assign(
            Load_Name=Load_Name,
            Date=lambda DF: pd.to_datetime('1899-12-30') + pd.to_timedelta(DF.Date, unit='D') if 'xlsb' in file_name else pd.to_datetime(DF.Date) # Adding this since pandas messes up reading dates from xlsb files sometimes
        )
    
    

In [ ]:
df_tags = pd.DataFrame()

for folder_name in folder_names:
    
    folder_path = os.path.join(monthly_data_folder_path, folder_name, 'CIEP') # Only aggregating CIEP data
    
    for file_name in os.listdir(folder_path):

        if 'PLA' in file_name: # Adding this filter only to aggregate tags
            
            if 'Recent' in folder_name:
                df_tags_temp = aggregate_tags(folder_path, file_name, 'excel') # Newer files in excel binary
            else:
                df_tags_temp = aggregate_tags(folder_path, file_name, 'csv') # Older files in csv
        
            df_tags = pd.concat([df_tags, df_tags_temp])

In [ ]:
df_tags_pivoted = df_tags.assign(
    Date=lambda DF: pd.to_datetime(DF.Date, errors='coerce') # There are some missing dates - hence coercing the errors and then interpolating
).interpolate().rename(
    columns={
        'Capacity Peak Load Allocation': 'PLC',
        'Transmission Peak Load Allocation': 'NSPL'
    }
).drop_duplicates().pivot_table( # Drops just 1 row in RECO Eligible
    index='Date',
    columns='Load_Name',
    values=['PLC', 'NSPL']
).pipe(
    lambda DF: DF[DF.index >= '2014-01-01'] # Filtering out data before 2014. Raw data starts from 2001-08-01
)

df_tags_pivoted

Fixing outliers in tags

In [ ]:
# df_tags_pivoted[('PLC', 'RECO_BGS')].loc[
#     lambda DF: DF.index == '2023-11-09'
# ] = 0.5 * df_tags_pivoted[('PLC', 'RECO_BGS')].loc[
#     lambda DF: DF.index == '2023-11-08'
# ] + 0.5 * df_tags_pivoted[('PLC', 'RECO_BGS')].loc[
#     lambda DF: DF.index == '2023-11-10'
# ]

df_tags_pivoted.loc['2023-11-09', ('PLC', 'RECO_BGS')] = (
    0.5 * df_tags_pivoted.loc['2023-11-08', ('PLC', 'RECO_BGS')] +
    0.5 * df_tags_pivoted.loc['2023-11-10', ('PLC', 'RECO_BGS')]
)

df_tags_pivoted.loc['2023-11-09', ('PLC', 'RECO_Eligible')] = (
    0.5 * df_tags_pivoted.loc['2023-11-08', ('PLC', 'RECO_Eligible')] +
    0.5 * df_tags_pivoted.loc['2023-11-10', ('PLC', 'RECO_Eligible')]
)

df_tags_pivoted.loc['2017-07-06', ('PLC', 'ACE_Eligible')] = (
    0.5 * df_tags_pivoted.loc['2017-07-05', ('PLC', 'ACE_Eligible')] +
    0.5 * df_tags_pivoted.loc['2017-07-07', ('PLC', 'ACE_Eligible')]
)

df_tags_pivoted.loc['2017-07-06', ('NSPL', 'ACE_Eligible')] = (
    0.5 * df_tags_pivoted.loc['2017-07-05', ('NSPL', 'ACE_Eligible')] +
    0.5 * df_tags_pivoted.loc['2017-07-07', ('NSPL', 'ACE_Eligible')]
)

In [ ]:
df_tags_pivoted = df_tags_pivoted.pipe(
    flatten_columns
).reindex(
    columns=['PLC_' + zone + '_BGS' for zone in zones_without_gen] 
    + ['NSPL_' + zone + '_BGS' for zone in zones_without_gen] 
    + ['PLC_' + zone + '_Eligible' for zone in zones_without_gen] 
    + ['NSPL_' + zone + '_Eligible' for zone in zones_without_gen]
)

df_tags_pivoted

In [ ]:
for zone in zones_without_gen:
    fig = px.line(
        df_tags_pivoted.reset_index(),
        x='Date',
        y=[f'PLC_{zone}_Eligible', f'NSPL_{zone}_Eligible'],
        title=f'PLC/NSPL_{zone}_Eligible tags'
    )

    fig.show()

PLC:

a. ACE, PSEG, RECO - Unscaled

b. JCP&L - Need to look into

NSPL

a. ACE, PSEG, RECO - Scaled (aka scaling factors = 1)

b. JCP&L - Need to look into

### Counts

In [ ]:
def aggregate_counts(file_path):
    
    if 'xlsb' in file_path:
        engine='pyxlsb' # More recent files in xlsb
    else:
        engine=None

    counts_data =  pd.read_excel(
        file_path,
        engine=engine,
        skiprows=4,
        usecols='B:I',
        sheet_name=None,
        nrows=5
    )

    # Adding this since the sheet and column names have spaces sometimes
    counts_data = {
        sheet.strip(): DF.rename(columns=str.strip) for sheet, DF in counts_data.items()
    }

    df_counts = pd.concat(counts_data, names=['Month', 'Row']).reset_index()

    return df_counts

In [ ]:
historical_switching_file_name = 'Hist_CIEP_Switching_Statistics.xlsx'

counts_historical_file_path = os.path.join(monthly_data_folder_path, switching_folder_name, historical_switching_file_name)

df_counts_historical = aggregate_counts(counts_historical_file_path)

# df_counts_historical

In [ ]:
recent_switching_file_name = 'CIEP_Switching_Historical_December_2025_Update.xlsb'

recent_counts_file_path = os.path.join(monthly_data_folder_path, recent_folder_name, recent_switching_file_name)

df_counts_recent = aggregate_counts(recent_counts_file_path)

# df_counts_recent

In [ ]:
df_counts = pd.concat([df_counts_historical, df_counts_recent])

df_counts

In [ ]:
df_counts_pivoted = df_counts.drop(
    columns=['Row', 'Unnamed: 5']
).rename(
    columns={
        'Unnamed: 1': 'Zone',
        'Total': 'Eligible_counts',
        'Switching': 'Switching_counts',
        'Percentage': 'Percentage_counts',
        'Total.1': 'Eligible_load',
        'Switching.1': 'Switching_load',
        'Percentage.1': 'Percentage_load'
    }
).assign(
    Month=lambda DF: pd.to_datetime(DF.Month, format='%B %Y'),
    Zone=lambda DF: DF.Zone.replace( 
        {
            'PSE&G*': 'PSE&G', # These difference are only in the very old data
            'Conectiv': 'ACE'
        }
    )
).pivot_table(
    index='Month',
    columns='Zone',
    values=['Eligible_counts', 'Switching_counts', 'Percentage_counts', 'Eligible_load', 'Switching_load', 'Percentage_load']
).pipe(
    lambda DF: DF[DF.index >= '2013-12-01'] # Filering out data before 2014. Raw data starts in August 2003
).interpolate()

df_counts_pivoted

In [ ]:
df_counts_pivoted.xs(
    'Eligible_counts', axis=1, level=0
).pipe(
    px.line
)

### Deration Factors

In [ ]:
df_deration_factors = pd.DataFrame()

for folder_name in folder_names:
    folder_path = os.path.join(monthly_data_folder_path, folder_name) 
    
    for file_name in os.listdir(folder_path):
        if ('Derating' in file_name) or ('De-Rating' in file_name):
            
            print(f'Aggregating deration factors from {file_name}')
            
            if 'csv' in file_name:
                df_deration_factors_temp = pd.read_csv(
                    os.path.join(folder_path, file_name),
                    skiprows=4
                ).assign(
                    Load_Name = file_name.split('_')[1],
                    Date=lambda DF: pd.to_datetime(DF.Date)
                ).rename(
                    columns={
                        'Loss De-Rating Factor ': 'Deration_Factor',
                        'HE': 'Hour'
                    }
                )
            
            elif 'xlsb' in file_name:
                
                df_deration_factors_temp = pd.read_excel(
                    os.path.join(folder_path, file_name),
                    engine='pyxlsb',
                    skiprows=5
                ).assign(
                    Load_Name = file_name.split('_')[0],
                    # Date=lambda DF: pd.to_datetime(DF.Date)
                    Date=lambda DF: pd.to_datetime('1899-12-30') + pd.to_timedelta(DF.Date, unit='D'), # Adding this to handle issues reading xlsb files
                ).rename(
                    columns={
                        'Loss De-Rating Factor ': 'Deration_Factor',
                        'HE': 'Hour'
                    }
                )                

            df_deration_factors = pd.concat([df_deration_factors, df_deration_factors_temp])

df_deration_factors

Doing the DST adjustments for these 3 zones. JCPL has some other issues so is treated separately.

In [ ]:
deration_factors_dict = {}

for zone in ['ACE', 'PSEG', 'RECO']:
    deration_factors_dict[zone] = df_deration_factors.loc[
        lambda DF: DF.Load_Name == zone
    ].loc[
        lambda DF: DF.Date >= '2014-01-01'
    ].drop(
        columns='Load_Name'
    ).set_index(
        ['Date', 'Hour']
    ).pipe(
        handle_dst_dates,
        dst_end_method='drop',
        he2_to_drop='first',
        verbose=True
    )

In [ ]:
df_deration_factors_pivoted = deration_factors_dict['ACE'].merge(
    deration_factors_dict['PSEG'],
    left_index=True,
    right_index=True
).merge(
    deration_factors_dict['RECO'],
    left_index=True,
    right_index=True    
).rename(
    columns={
        'Deration_Factor_x': 'ACE',
        'Deration_Factor_y': 'PSEG',
        'Deration_Factor': 'RECO'
    }
)

df_deration_factors_pivoted

In [ ]:
df_deration_factors_pivoted = df_deration_factors_pivoted.merge(df_deration_factors.loc[
        lambda DF: DF.Load_Name == 'JCP&L'
    ].loc[
        lambda DF: DF.Date >= '2014-01-01'
    ].drop(
        columns='Load_Name'
    ).assign(
        Date=lambda DF: pd.to_datetime(DF.Date.dt.date)
    ).set_index(
        ['Date', 'Hour']
    ).pipe(
        handle_dst_dates,
        dst_end_method='use_he_25',
        # he2_to_drop='first',
        verbose=True
    ),
    left_index=True,
    right_index=True
    ).rename(
        columns={
            'Deration_Factor': 'JCP&L'
        }
    ).reindex(
        columns=['ACE', 'JCP&L', 'PSEG', 'RECO']
    )

df_deration_factors_pivoted

In [ ]:
# Followed this approach originally

# df_deration_factors_pivoted = df_deration_factors.pivot_table( # Using pivot table to groupby since there are some duplicate entries 
#     index=['Date', 'Hour'],
#     columns='Load_Name',
#     values='Deration_Factor'
# ).pipe(
#     lambda DF: DF[DF.index.get_level_values(0) >= '2014-01-01'] # filtering out historical data before 2014-01-01
# ).reindex(
#     hourly_index('2014-01-01', '2025-11-30') # Doing this since there are some HE 25 and missing data. May have to update the end date if more data is posted
# ).interpolate()

# df_deration_factors_pivoted

In [ ]:
df_deration_factors_pivoted.reset_index().pipe(
    px.line,
    x='Date',
    y=zones_without_gen
)

### Scaling Factors

In [ ]:
df_scaling_factors = pd.DataFrame()

for folder_name in folder_names:
    
    folder_path = os.path.join(monthly_data_folder_path, folder_name) 
    
    for file_name in os.listdir(folder_path):
        
        if 'Scaling' in file_name:

            print(f'Aggregating scaling factors from {file_name}')

            zone = file_name.split('_')[0]

            if zone in ['ACE', 'PSEG', 'RECO']: # For these zones, the NSPL scaling factor is 1
                df_scaling_factors_temp = pd.read_excel(
                    os.path.join(folder_path, file_name),
                    engine='pyxlsb',
                    skiprows=5
                ).assign(
                    Load_Name=zone,
                    Type='PLC'   
                )

                df_scaling_factors = pd.concat([df_scaling_factors, df_scaling_factors_temp])

            else: # JCPL
                df_scaling_factors_temp_1 = pd.read_excel(
                    os.path.join(folder_path, file_name),
                    engine='pyxlsb',
                    skiprows=5,
                    sheet_name='CAP DZSF'
                ).assign(
                    Load_Name=zone,
                    Type='PLC'
                )

                df_scaling_factors_temp_2 = pd.read_excel(
                    os.path.join(folder_path, file_name),
                    engine='pyxlsb',
                    skiprows=5,
                    sheet_name='NSPL DZSF'
                ).assign(
                    Load_Name=zone,
                    Type='NSPL'
                )

                df_scaling_factors = pd.concat([df_scaling_factors, df_scaling_factors_temp_1, df_scaling_factors_temp_2])

Pulling RECO NSPL SF from internal file since the data is not published

In [ ]:
internal_dzfs_path = r'K:\Valuation\MODELS\VQSWAP\VQSwap Update Info\PJM\Capacity\Zonal Scaling Factors\PJM Daily Zonal Scaling Factors v3.xlsb'

df_nspl_reco_scaling_factors = pd.read_excel(
    internal_dzfs_path,
    sheet_name='NSPL DZSF',
    skiprows=1,
    usecols='B:O'
).assign(
    Date=lambda DF: pd.to_datetime(DF.Date, origin='1899-12-30', unit='D')
)[['Date', 'RECO']].set_index(
    'Date'
).set_axis(
    pd.MultiIndex.from_tuples(
    [('NSPL', 'RECO')]
), 
axis='columns'
)

In [ ]:
df_scaling_factors_pivoted = df_scaling_factors.drop(
    columns=['Unnamed: 2', 'Unnamed: 3']
).assign(
    Date=lambda DF: pd.to_datetime(DF.Date, unit='D', origin='1899-12-30') # Converting excel date to datetime object
).pivot(
    index='Date',
    columns=['Type', 'Load_Name'],
    values='DZSF'
).pipe(
    lambda DF: DF[DF.index >= '2014-01-01'] # Filtering out data before 2014. Raw data starts from 2011-06-01
).join(
    df_nspl_reco_scaling_factors
).replace(
    np.nan, 1 # filling the missing NSPL scaling factors for JCP&L and RECO
)

df_scaling_factors_pivoted

In [ ]:
df_scaling_factors_pivoted.pipe(
    flatten_columns
).pipe(
    px.line
) 

### Checking internal tags to see if they are scaled or unscaled

Internal data pulled using "K:\Valuation\_Analysts\Hemanth\Python Notebooks\Deals\Download-MSRS-Reports.ipynb"

Note: 

BGS-Total = BGS-CIEP + BGS-RSCP

Eligible-Total = Eligible-CIEP + Eligible-RSCP. I think Retail-Total is the same as Eligible-Total

#### JCP&L

In [ ]:
msrs_folder_path = r"K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Data\msrs_reports_recon_jcpl"

df_internal_tags = pd.DataFrame()

for file_name in os.listdir(msrs_folder_path):
    if ('Locational Reliability Charge Summary' in file_name) or ('NITS Charge Summary' in file_name):
        
        print(f'Aggregating internal tags from {file_name}')
        
        df_internal_tags_temp = pd.read_csv(
            os.path.join(msrs_folder_path, file_name),
            skiprows=4
        )

        df_internal_tags = pd.concat([df_internal_tags, df_internal_tags_temp])

In [ ]:
# Pulling appropriate RR from capacity forecast file

rr_jcpl_py_20_21, rr_jcpl_py_21_22 = pd.read_excel(
    r'K:\Valuation\MODELS\VQSWAP\VQSwap Update Info\PJM\Capacity\Capacity FORECAST ALL - CURRENT.xlsx',
    sheet_name='Sheet1',
    skiprows=24,
    usecols='A:U'
).dropna(how='all').iloc[
    10, [14, 18]
].values

In [ ]:
df_internal_tags[
    ['Date', 'UCAP Obligation (MW)', 'Daily Peak Load (MW)']
].dropna(how='all').assign(
    Date=lambda DF: pd.to_datetime(DF.Date)
).sort_values(
    by='Date'
).assign(
    Planning_year=lambda DF: DF.Date.apply(planning_year),
    RR=lambda DF: DF.apply(lambda DF: rr_jcpl_py_20_21 if DF.Planning_year == '2020-2021' else rr_jcpl_py_21_22, axis=1),
    percent_served = lambda DF: DF.apply(lambda DF: 1 / 6 if DF.Planning_year == '2020-2021' else 0.3, axis=1), # These percentages served are exact and have been verified from prior workups
    Internal_PLC_Scaled=lambda DF: DF['UCAP Obligation (MW)'] / (DF.RR * DF.percent_served),
    Internal_NSPL_Scaled=lambda DF: DF['Daily Peak Load (MW)'] / DF.percent_served
).merge(
    df_tags_pivoted[['PLC_JCP&L_BGS', 'NSPL_JCP&L_BGS']].reset_index(),
on='Date'
).pipe(
    px.scatter,
    x='Date',
    # y=['Internal_PLC_Scaled', 'PLC_JCP&L_BGS'],
    y=['Internal_NSPL_Scaled', 'NSPL_JCP&L_BGS'],
    title='Internal vs posted NSPL comparison'
)

#### PSEG

In [ ]:
msrs_folder_path = r"K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Data\msrs_reports_recon_pseg"

df_internal_tags = pd.DataFrame()

for file_name in os.listdir(msrs_folder_path):
    if ('Locational Reliability Charge Summary' in file_name) or ('NITS Charge Summary' in file_name):
        
        print(f'Aggregating internal tags from {file_name}')
        
        df_internal_tags_temp = pd.read_csv(
            os.path.join(msrs_folder_path, file_name),
            skiprows=4
        )

        df_internal_tags = pd.concat([df_internal_tags, df_internal_tags_temp])

In [ ]:
# Pulling appropriate RR from capacity forecast file

rr_pseg_py_20_21, rr_pseg_py_21_22, rr_pseg_py_22_23, rr_pseg_py_23_24, rr_pseg_py_24_25 = pd.read_excel(
    r'K:\Valuation\MODELS\VQSWAP\VQSwap Update Info\PJM\Capacity\Capacity FORECAST ALL - CURRENT.xlsx',
    sheet_name='Sheet1',
    skiprows=24,
    usecols='A:U'
).dropna(how='all').iloc[
    15, 14:19
].values

In [ ]:
# helper functions

def pseg_rr(DF):
    if DF.Planning_year == '2020-2021':
        return rr_pseg_py_20_21
    elif DF.Planning_year == '2021-2022':
        return rr_pseg_py_21_22
    elif DF.Planning_year == '2022-2023':
        return rr_pseg_py_22_23
    elif DF.Planning_year == '2023-2024':
        return rr_pseg_py_23_24
    elif DF.Planning_year == '2024-2025':
        return rr_pseg_py_24_25
    
def pseg_percent_served(DF):
    if DF.Planning_year == '2020-2021':
        return 6 / 24
    elif DF.Planning_year == '2021-2022':
        return 10 / 24
    elif DF.Planning_year == '2022-2023':
        return 10 / 22
    elif DF.Planning_year == '2023-2024':
        return 2 / 22
    elif DF.Planning_year == '2024-2025':
        return 3 / 21

In [ ]:
df_internal_tags[
    ['Date', 'UCAP Obligation (MW)', 'Daily Peak Load (MW)']
].dropna(how='all').assign(
    Date=lambda DF: pd.to_datetime(DF.Date)
).sort_values(
    by='Date'
).assign(
    Planning_year=lambda DF: DF.Date.apply(planning_year),
    RR=lambda DF: DF.apply(pseg_rr, axis=1),
    percent_served = lambda DF: DF.apply(pseg_percent_served, axis=1), # These percentages served are exact and have been verified from prior workups
    Internal_PLC_Scaled=lambda DF: DF['UCAP Obligation (MW)'] / (DF.RR * DF.percent_served),
    Internal_NSPL_Scaled=lambda DF: DF['Daily Peak Load (MW)'] / DF.percent_served
).merge(
    df_tags_pivoted[['PLC_PSEG_BGS', 'NSPL_PSEG_BGS']].reset_index(),
on='Date'
).pipe(
    px.scatter,
    x='Date',
    y=['Internal_PLC_Scaled', 'PLC_PSEG_BGS'],
    # y=['Internal_NSPL_Scaled', 'NSPL_PSEG_BGS'],
    title='Internal vs posted PLC comparison'
)

#### RECO

In [ ]:
msrs_folder_path = r"K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Data\msrs_reports_recon_reco"

df_internal_tags = pd.DataFrame()

for file_name in os.listdir(msrs_folder_path):
    if ('Locational Reliability Charge Summary' in file_name) or ('NITS Charge Summary' in file_name):
        
        print(f'Aggregating internal tags from {file_name}')
        
        df_internal_tags_temp = pd.read_csv(
            os.path.join(msrs_folder_path, file_name),
            skiprows=4
        )

        df_internal_tags = pd.concat([df_internal_tags, df_internal_tags_temp])

In [ ]:
rr_reco_22_23 = pd.read_excel(
    r'K:\Valuation\MODELS\VQSWAP\VQSwap Update Info\PJM\Capacity\Capacity FORECAST ALL - CURRENT.xlsx',
    sheet_name='Sheet1',
    skiprows=24,
    usecols='A:U'
).dropna(how='all').iloc[
    16, 16
]

In [ ]:
df_internal_tags[
    ['Date', 'UCAP Obligation (MW)', 'Daily Peak Load (MW)']
].dropna(how='all').assign(
    Date=lambda DF: pd.to_datetime(DF.Date)
).sort_values(
    by='Date'
).assign(
    Planning_year=lambda DF: DF.Date.apply(planning_year),
    RR=lambda DF: rr_reco_22_23,
    percent_served = 1, # These percentages served are exact and have been verified from prior workups
    Internal_PLC_Scaled=lambda DF: DF['UCAP Obligation (MW)'] / (DF.RR * DF.percent_served),
    Internal_NSPL_Scaled=lambda DF: DF['Daily Peak Load (MW)'] / DF.percent_served
).merge(
    df_tags_pivoted[['PLC_RECO_BGS', 'NSPL_RECO_BGS']].reset_index(),
on='Date'
).pipe(
    px.scatter,
    x='Date',
    # y=['Internal_PLC_Scaled', 'PLC_RECO_BGS'],
    y=['Internal_NSPL_Scaled', 'NSPL_RECO_BGS'],
    title='Internal vs posted NSPL comparison'
)

### Doing some checks on the tags to see if they sum up correctly to the totals

Conclusion of all this analysis: Hi CIEP team. I have done the following checks on the tags (in addition to the checks on JCPL tags we did above).
 
The BGS CIEP and BGS RSCP tags sum up to the BGS TOTAL tags (makes sense)
The BGS CIEP Eligible tags and the BGS RSCP Eligible tags sum up to the Retail TOTAL tags without scaling. (This means that their claim that the BGS CIEP Eligible tags need to be scaled is incorrect - this seems to have been documented by Kevin O as well last time).
So I propose we treat the tags in the following way:
 
ACE, RECO, PSEG - no ambiguity - PLC unscaled and NSPL scaled (both BGS and Eligible)
 
JCPL - 
BGS PLC unscaled (matches what they say), 
Eligible PLC already scaled (contrary to what they say). 
BGS NSPL unscaled (matches what they say), 
Eligible NSPL already scaled (contrary to what they say)
 
Is this fair?
 

Aggregating RSCP and TOTAL tags and pivoting to the required format

In [ ]:
df_rscp_tags = pd.DataFrame()
df_total_tags = pd.DataFrame()

for folder_name in folder_names_without_recent:
    
    folder_path = os.path.join(monthly_data_folder_path, folder_name, 'RSCP') # Aggregating RSCP data
    
    for file_name in os.listdir(folder_path):

        if 'PLA' in file_name: # Adding this filter only to aggregate tags
            
            if 'Recent' in folder_name:
                df_rscp_tags_temp = aggregate_tags(folder_path, file_name, 'excel') # Newer files in excel binary
            else:
                df_rscp_tags_temp = aggregate_tags(folder_path, file_name, 'csv') # Older files in csv
        
            df_rscp_tags = pd.concat([df_rscp_tags, df_rscp_tags_temp])

    folder_path = os.path.join(monthly_data_folder_path, folder_name, 'TOTAL') # Aggregating RSCP data
    
    for file_name in os.listdir(folder_path):

        if 'PLA' in file_name: # Adding this filter only to aggregate tags
            
            if 'Recent' in folder_name:
                df_total_tags_temp = aggregate_tags(folder_path, file_name, 'excel') # Newer files in excel binary
            else:
                df_total_tags_temp = aggregate_tags(folder_path, file_name, 'csv') # Older files in csv
        
            df_total_tags = pd.concat([df_total_tags, df_total_tags_temp])

In [ ]:
df_rscp_tags_pivoted = df_rscp_tags.assign(
    Date=lambda DF: pd.to_datetime(DF.Date, errors='coerce') # There are some missing dates - hence coercing the errors and then interpolating
).interpolate().rename(
    columns={
        'Capacity Peak Load Allocation': 'PLC',
        'Transmission Peak Load Allocation': 'NSPL'
    }
).drop_duplicates().pivot_table( # Drops just 1 row in RECO Eligible
    index='Date',
    columns='Load_Name',
    values=['PLC', 'NSPL']
).pipe(
    lambda DF: DF[DF.index >= '2014-01-01'] # Filtering out data before 2014. Raw data starts from 2001-08-01
)

df_rscp_tags_pivoted

In [ ]:
df_total_tags_pivoted = df_total_tags.assign(
    Date=lambda DF: pd.to_datetime(DF.Date, errors='coerce') # There are some missing dates - hence coercing the errors and then interpolating
).interpolate().rename(
    columns={
        'Capacity Peak Load Allocation': 'PLC',
        'Transmission Peak Load Allocation': 'NSPL'
    }
).drop_duplicates().pivot_table( # Drops just 1 row in RECO Eligible
    index='Date',
    columns='Load_Name',
    values=['PLC', 'NSPL']
).pipe(
    lambda DF: DF[DF.index >= '2014-01-01'] # Filtering out data before 2014. Raw data starts from 1999-11-01
)

df_total_tags_pivoted

Adding CIEP and RSCP Eligible tags and comparing to the total eligible/retail. Conclusion - tie

In [ ]:
print(f'This is a check on the NSPL tags: \n')

for zone in zones_without_gen:

    print(f'Checking if the sum of CIEP and RSCP Eligible NSPL tags sum up to the TOTAL RETAIL and printing the min, max and mean of the delta for zone {zone} \n')

    print(df_tags_pivoted.loc[
        lambda DF: DF.index <= '2022-05-31'
    ][['NSPL_' + zone + '_Eligible']].rename(
        columns={
            'NSPL_' + zone + '_Eligible': 'NSPL_' + zone + '_Eligible_CIEP'
        }
    ).merge(
        right=df_rscp_tags_pivoted.pipe(
        flatten_columns
    )[['NSPL_' + zone + '_Eligible']].rename(
        columns={
            'NSPL_' + zone + '_Eligible': 'NSPL_' + zone + '_Eligible_RSCP'
        }
    ),
    on='Date'
    ).merge(
        right=df_total_tags_pivoted.pipe(
        flatten_columns
    )[['NSPL_' + zone + '_Retail']],
    on='Date'
    ).assign(
        Check=lambda DF: DF['NSPL_' + zone + '_Eligible_CIEP'] + DF['NSPL_' + zone + '_Eligible_RSCP'] - DF['NSPL_' + zone + '_Retail']
    ).Check.describe().loc[['min', 'max', 'mean']])

    print('\n')

In [ ]:
print(f'This is a way to check on the PLC tags: \n')

for zone in zones_without_gen:

    print(f'Checking if the sum of CIEP and RSCP Eligible PLC tags sum up to the TOTAL RETAIL and printing the min, max and mean of the delta for zone {zone} \n')

    print(df_tags_pivoted.loc[
        lambda DF: DF.index <= '2022-05-31'
    ][['PLC_' + zone + '_Eligible']].rename(
        columns={
            'PLC_' + zone + '_Eligible': 'PLC_' + zone + '_Eligible_CIEP'
        }
    ).merge(
        right=df_rscp_tags_pivoted.pipe(
        flatten_columns
    )[['PLC_' + zone + '_Eligible']].rename(
        columns={
            'PLC_' + zone + '_Eligible': 'PLC_' + zone + '_Eligible_RSCP'
        }
    ),
    on='Date'
    ).merge(
        right=df_total_tags_pivoted.pipe(
        flatten_columns
    )[['PLC_' + zone + '_Retail']],
    on='Date'
    ).assign(
        Check=lambda DF: DF['PLC_' + zone + '_Eligible_CIEP'] + DF['PLC_' + zone + '_Eligible_RSCP'] - DF['PLC_' + zone + '_Retail']
    ).Check.describe().loc[['min', 'max', 'mean']])

    print('\n')

Adding CIEP and RSCP BGS tags and comparing to the total BGS - Conclusion - tie

In [ ]:
for zone in zones_without_gen:

    print(f'Checking if the sum of CIEP and RSCP BGS NSPL tags sum up to the TOTAL BGS and printing the min, max and mean of the delta for zone {zone} \n')

    print(df_tags_pivoted.loc[
        lambda DF: DF.index <= '2022-05-31'
    ][['NSPL_' + zone + '_BGS']].rename(
        columns={
            'NSPL_' + zone + '_BGS': 'NSPL_' + zone + '_BGS_CIEP'
        }
    ).merge(
        right=df_rscp_tags_pivoted.pipe(
        flatten_columns
    )[['NSPL_' + zone + '_BGS']].rename(
        columns={
            'NSPL_' + zone + '_BGS': 'NSPL_' + zone + '_BGS_RSCP'
        }
    ),
    on='Date'
    ).merge(
        right=df_total_tags_pivoted.pipe(
        flatten_columns
    )[['NSPL_' + zone + '_Total']],
    on='Date'
    ).assign(
        Check=lambda DF: DF['NSPL_' + zone + '_BGS_CIEP'] + DF['NSPL_' + zone + '_BGS_RSCP'] - DF['NSPL_' + zone + '_Total']
    ).Check.describe().loc[['min', 'max', 'mean']])

    print('\n')

In [ ]:
for zone in zones_without_gen:

    print(f'Checking if the sum of CIEP and RSCP BGS PLC tags sum up to the TOTAL BGS and printing the min, max and mean of the delta for zone {zone} \n')

    print(df_tags_pivoted.loc[
        lambda DF: DF.index <= '2022-05-31'
    ][['PLC_' + zone + '_BGS']].rename(
        columns={
            'PLC_' + zone + '_BGS': 'PLC_' + zone + '_BGS_CIEP'
        }
    ).merge(
        right=df_rscp_tags_pivoted.pipe(
        flatten_columns
    )[['PLC_' + zone + '_BGS']].rename(
        columns={
            'PLC_' + zone + '_BGS': 'PLC_' + zone + '_BGS_RSCP'
        }
    ),
    on='Date'
    ).merge(
        right=df_total_tags_pivoted.pipe(
        flatten_columns
    )[['PLC_' + zone + '_Total']],
    on='Date'
    ).assign(
        Check=lambda DF: DF['PLC_' + zone + '_BGS_CIEP'] + DF['PLC_' + zone + '_BGS_RSCP'] - DF['PLC_' + zone + '_Total']
    ).Check.describe().loc[['min', 'max', 'mean']])

    print('\n')

### Unscaling JCPL tags (and RECO NSPLs)

In [ ]:
df_tags_pivoted_unscaled = df_tags_pivoted.merge(
    right=df_scaling_factors_pivoted.pipe(flatten_columns),
    on='Date'
)

df_tags_pivoted_unscaled

Unscaling BGS PLC and NSPL tags before 2017-06-01 and eligible tags before 2022-09-01

In [ ]:
df_tags_pivoted_unscaled['PLC_JCP&L_BGS'] = np.where(
    df_tags_pivoted_unscaled.index < '2017-06-01',
    df_tags_pivoted_unscaled['PLC_JCP&L_BGS'] / df_tags_pivoted_unscaled['PLC_JCP&L'],
    df_tags_pivoted_unscaled['PLC_JCP&L_BGS']
)

df_tags_pivoted_unscaled['NSPL_JCP&L_BGS'] = np.where(
    df_tags_pivoted_unscaled.index < '2017-06-01',
    df_tags_pivoted_unscaled['NSPL_JCP&L_BGS'] / df_tags_pivoted_unscaled['NSPL_JCP&L'],
    df_tags_pivoted_unscaled['NSPL_JCP&L_BGS']
)

df_tags_pivoted_unscaled['PLC_JCP&L_Eligible'] = np.where(
    df_tags_pivoted_unscaled.index < '2022-09-01',
    df_tags_pivoted_unscaled['PLC_JCP&L_Eligible'] / df_tags_pivoted_unscaled['PLC_JCP&L'],
    df_tags_pivoted_unscaled['PLC_JCP&L_Eligible']
)

df_tags_pivoted_unscaled['NSPL_JCP&L_Eligible'] = np.where(
    df_tags_pivoted_unscaled.index < '2022-09-01',
    df_tags_pivoted_unscaled['NSPL_JCP&L_Eligible'] / df_tags_pivoted_unscaled['NSPL_JCP&L'],
    df_tags_pivoted_unscaled['NSPL_JCP&L_Eligible']
)

df_tags_pivoted_unscaled['NSPL_RECO_BGS'] = df_tags_pivoted_unscaled['NSPL_RECO_BGS'] / df_tags_pivoted_unscaled['NSPL_RECO']
df_tags_pivoted_unscaled['NSPL_RECO_Eligible'] = df_tags_pivoted_unscaled['NSPL_RECO_Eligible'] / df_tags_pivoted_unscaled['NSPL_RECO'] 

In [ ]:
df_tags_pivoted_unscaled.drop( # dropping the scaling factors
    columns=['PLC_ACE', 'PLC_JCP&L', 'NSPL_JCP&L', 'PLC_PSEG', 'PLC_RECO', 'NSPL_RECO'],
    inplace=True
)

### Load Factor Analysis

The load factor is the most important analysis on this deal. Load factor = load divided by tag. For our usual deals, we prefer high load factor customers (high load, low tag) since we get paid more for more load, and incur lower costs on lower capacity.

However, CIEP is effectively a REC, Ancillary and DART deal that is paid on capacity. So low load factor customers (low load, high tag) since our REC, ancillary and DART costs will be lower while we get paid more for the UCAP.

The same load factor risk exists in all deals, but is more pronounced for this deal.

#### Setting up

In [ ]:
# Creating MultiIndex for load and tag for ease of calculation - should have ideally done this at the beginning

df_tags_pivoted_unscaled_multi = df_tags_pivoted_unscaled.copy()

df_tags_pivoted_unscaled_multi.columns = pd.MultiIndex.from_tuples([col.split('_') for col in df_tags_pivoted_unscaled.columns])

df_tags_pivoted_unscaled_multi

In [ ]:
# Dropping these columns as we already concluded that excess generation is not part of the pricing load and hence not used for anything

df_load_pivoted_multi = df_load_pivoted.copy().drop(
    columns=['JCP&L_Excess_Generation_BGS', 'JCP&L_Excess_Generation_Eligible', 'JCP&L_Net_BGS', 'JCP&L_Net_Eligible']
)

df_load_pivoted_multi.columns = pd.MultiIndex.from_tuples([col.split('_') for col in df_load_pivoted_multi.columns])

df_load_pivoted_multi

In [ ]:
# These are the term load factors implied from our current load and capacity forecasts - this will be overlain on the charts for context

chosen_lf = {
    'ACE': 0.7876,
    'JCP&L': 1.0191,
    'PSEG': 0.8887,
    'RECO': 0.5943
}

#### Daily load factor

In [ ]:
for zone in zones_without_gen:

    fig = (df_load_pivoted_multi.reset_index().groupby(
        [('Date', '')]
    ).mean().drop(
        columns=[('Hour', '')]
    ) / df_tags_pivoted_unscaled_multi.xs(
        'PLC', level=0, axis=1
    )).xs(
        zone, level=0, axis=1
    ).pipe(
        px.line,
        markers=True,
        title=f'{zone} Daily Load Factor',
        labels={
            'index': 'Date',
            'value': 'Load Factor'
        }
    )

    fig.add_hline(y=chosen_lf[zone], line_dash='dash', line_color='black')

    fig.show()

In [ ]:
for type in ['BGS', 'Eligible']:

    (df_load_pivoted_multi.reset_index().groupby(
            [('Date', '')]
        ).mean().drop(
            columns=[('Hour', '')]
        ) / df_tags_pivoted_unscaled_multi.xs(
            'PLC', level=0, axis=1
        )).xs(
            type, level=1, axis=1
        ).reset_index().rename(
            columns={
                'index': 'Date'
            }
        ).melt(
            id_vars='Date',
            var_name='Zone',
            value_name='Load_Factor'
        ).assign(
            Month=lambda DF: DF.Date.dt.month,
            Season=lambda DF: np.where(DF.Month.isin([12, 1, 2, 3]), 'Winter', np.where(DF.Month.isin([6, 7, 8, 9]), 'Summer', 'Shoulder'))
        ).pipe(
                px.histogram,
                x='Load_Factor',
                color='Season',
                facet_row='Zone',
                opacity=0.75,
                barmode='overlay',
                height=750,
                category_orders={'Season': ['Shoulder', 'Winter', 'Summer']},
                title=f'{type} daily load factor histogram'
            ).add_vline(
                x=chosen_lf['ACE'], line_dash='dash', line_color='black', row=4
            ).add_vline(
                x=chosen_lf['JCP&L'], line_dash='dash', line_color='black', row=3
            ).add_vline(
                x=chosen_lf['PSEG'], line_dash='dash', line_color='black', row=2
            ).add_vline(
                x=chosen_lf['RECO'], line_dash='dash', line_color='black', row=1
            ).show()

#### Monthly load factor

In [ ]:
monthly_load_factor = (df_load_pivoted_multi.reset_index().assign(
        Month_start=lambda DF: DF[('Date', '')].dt.to_period('M').dt.to_timestamp()
    ).groupby(
        [('Month_start', '')]
    ).mean().drop(
        columns=[
            ('Date', ''),
            ('Hour', '')
        ]
    ) / df_tags_pivoted_unscaled_multi.reset_index().assign(
        Month_start=lambda DF: DF[('Date', '', '')].dt.to_period('M').dt.to_timestamp()
    ).groupby(
        [('Month_start', '', '')]
    ).mean().drop(
        columns=[
            ('Date', '', '')
        ]
    ).xs(
        'PLC', level=0, axis=1
    ).rename_axis([
        ('Month_start', '')
    ]))

for zone in zones_without_gen:

    fig = monthly_load_factor.reset_index().pipe(
        flatten_columns
    ).pipe(
        px.line,
        x='Month_start',
        y=[zone + '_BGS', zone + '_Eligible'],
        markers=True,
        title=f'{zone} Monthly Load Factor',
        labels={
            'value': 'Load Factor'
        }
    )

    fig.add_hline(
        y=chosen_lf[zone],
        line_dash='dash',
        line_color='black'
    )

    fig.show()


#### Monthly load factor box plots

In [ ]:
for zone in zones_without_gen:

    fig = monthly_load_factor.xs(
        'BGS', level=1, axis=1
    ).reset_index().rename(
        columns={
            ('Month_start', '') : 'Month_start'
        }
    ).assign(
        PY=lambda DF: DF.Month_start.apply(planning_year)
    ).pipe(
        px.box,
        x='PY',
        y=zone,
        points='all',
        title=f'{zone} monthly load factor distribution'
    )

    fig.add_hline(
        y=chosen_lf[zone],
        line_dash='dash',
        line_color='red'
    )

    fig.show()


In [ ]:
monthly_load_factor.xs(
        'BGS', level=1, axis=1
    ).reset_index().rename(
        columns={
            ('Month_start', '') : 'Month_start'
        }
    ).assign(
        PY=lambda DF: DF.Month_start.apply(planning_year)
    ).drop(
        columns='Month_start'
    ).groupby(
        'PY'
    ).mean().reset_index().melt(
        id_vars='PY',
        value_vars=zones_without_gen,
        var_name='Zone',
        value_name='Load Factor'
    ).pipe(
        px.line,
        x='PY',
        y='Load Factor',
        facet_row='Zone',
        markers='True',
        height=800,
        title='Average load factor by planning year'
    ).add_hline(
        y=chosen_lf['ACE'], line_dash='dash', line_color='red', row=4
    ).add_hline(
        y=chosen_lf['JCP&L'], line_dash='dash', line_color='red', row=3
    ).add_hline(
        y=chosen_lf['PSEG'], line_dash='dash', line_color='red', row=2
    ).add_hline(
        y=chosen_lf['RECO'], line_dash='dash', line_color='red', row=1
    ).update_yaxes(matches=None)

#### Plotting load and tag separately as a sanity check

In [ ]:
def divide_load_by_tags(DF, zones):
    for zone in zones:
        DF[('Load_Factor', zone)] = DF[('Load', zone)] / DF[('Tags', zone)]
    return DF


The long pipe in the next cell is just for merging the load and tags so that the load factor can be plotted in the same chart as load and tag

In [ ]:
for zone in zones_without_gen:

    df_load_pivoted_multi.reset_index().assign(
        Month_start=lambda DF: DF[('Date', '')].dt.to_period('M').dt.to_timestamp()
    ).groupby(
        [('Month_start', '')]
    ).mean().drop(
        columns=[
            ('Date', ''),
            ('Hour', '')
        ]
    ).xs(
        'BGS', level=1, axis=1
    ).reset_index().rename(
        columns={
            ('Month_start', ''): 'Month_start'
        }
    ).set_index('Month_start').set_axis(
        pd.MultiIndex.from_product(
        [
            ['Load'], zones_without_gen
        ]
    ), axis=1  
        ).merge(
    df_tags_pivoted_unscaled_multi.reset_index().assign(
            Month_start=lambda DF: DF[('Date', '', '')].dt.to_period('M').dt.to_timestamp()
        ).groupby(
            [('Month_start', '', '')]
        ).mean().drop(
            columns=[
                ('Date', '', '')
            ]
        ).xs(
            'PLC', level=0, axis=1
        ).xs(
            'BGS', level=1, axis=1
        ).reset_index().rename(
            columns={
            ('Month_start', '', ''): 'Month_start' 
            }
        ).set_index('Month_start').set_axis(
                pd.MultiIndex.from_product(
        [
            ['Tags'], zones_without_gen
        ]
    ), axis=1  
        ),
        left_index=True,
        right_index=True
        ).pipe(
            divide_load_by_tags, zones_without_gen
        ).xs(
            zone, level=1, axis=1
        ).reset_index().melt(
            id_vars='Month_start',
            value_vars=['Load', 'Tags', 'Load_Factor']
        ).pipe(
            px.line,
            x='Month_start',
            y='value',
            facet_row='variable',
            height=750,
            title=f'{zone} monthly load, tag and load factor',
            markers=True
        ).update_yaxes(matches=None).for_each_annotation(
            lambda txt: txt.update(text=txt.text.split("=")[-1])
        ).add_hline(
            y=chosen_lf[zone], line_dash='dash', line_color='red', row=1
        ).show()
    


#### Plotting historical CP charts for capacity (using BIG migration-adjusted load)

In [ ]:
peak_dates = pd.read_excel(
    r'K:\Valuation\MODELS\VQSWAP\VQSwap Update Info\PJM\Capacity\Capacity FORECAST ALL - CURRENT.xlsx',
    sheet_name='Zonal Peaks',
    usecols=range(10, 22),
    skiprows=2,
    nrows=6
)

peak_dates

In [ ]:
peak_hours = pd.read_excel(
    r'K:\Valuation\MODELS\VQSWAP\VQSwap Update Info\PJM\Capacity\Capacity FORECAST ALL - CURRENT.xlsx',
    sheet_name='Zonal Peaks',
    usecols=range(10, 22),
    skiprows=8,
    nrows=6 
)

peak_hours.columns = peak_dates.columns

peak_hours

In [ ]:
peak_date_hours = (peak_dates.astype(str) + ' ' + (peak_hours - 1).astype(str) + ':00:00').apply(pd.to_datetime)

peak_date_hours = peak_date_hours.melt().rename(
    columns={
        'variable': 'PY',
        'value': 'DateTime'
    }
)

peak_date_hours

In [ ]:
load_template_path = r'K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Load\Pri Hemanth BGS-CIEP 2026-02 Load Template Pro v2.1.xlsm'

for sheet_name in [
    'AECO_ACE_BGS',
    'JCPL_JCPL_BGS',
    'PSEG_PSEG_BGS',
    'RECO_RECO_BGS'
]:

    df_temp = pd.read_excel(
        load_template_path,
        sheet_name=sheet_name,
        usecols=[1, 2, 6],
        skiprows=6
    ).rename(
        columns={
            'Big.1': 'Load (MWh)'
        }
    ).assign(
        # DateTime=lambda DF: DF.Date + pd.to_timedelta(DF.HE - 1, unit='h'),
        Year=lambda DF: DF.Date.dt.year
    )

    fig = peak_date_hours.assign(
    Date=lambda DF: pd.to_datetime(DF.DateTime.dt.date)
    ).merge(
        df_temp,
        on='Date'
    ).pipe(
        px.line,
        x='HE',
        y='Load (MWh)',
        facet_col='Year',
        facet_col_wrap=4,
        color='Date',
        title=f'{sheet_name.split('_')[0]} 5CP Days'
        # markers=True
    )

    count = 0

    years = [2022, 2023, 2024, 2025, 2018, 2019, 2020, 2021, 2014, 2015, 2016, 2017] # Setting this to be the order since plotly sets row 1, col 1 to be the the bottom left

    for row in range(3): # 12 years in total = 3 * 4
        for col in range(4):

            peaks = peak_date_hours.assign(
                    Date=lambda DF: pd.to_datetime(DF.DateTime.dt.date)
                    ).merge(
                        df_temp,
                        on='Date'
                    ).assign(
                        DateTime_Check=lambda DF: DF.Date + pd.to_timedelta(DF.HE, unit='h')
                    ).loc[
                        lambda DF: DF.DateTime == DF.DateTime_Check
                    ][['HE', 'Load (MWh)', 'Year']].loc[
                        lambda DF: DF.Year == years[count]
                    ]
            
            cp_peak_hours = peaks['HE']
            cp_peak_load = peaks['Load (MWh)']

            fig.add_scatter(
                x=cp_peak_hours,
                y=cp_peak_load,
                mode='markers',
                marker=dict(
                    size=7,
                    color='black'
                ),
                row=row + 1,
                col=col + 1
            )

            count += 1

    fig.show()

### DART spread analysis

In [ ]:
dart_zones = {
    'AECO': 116472927,
    'JCPL': 116472945,
    'PSEG': 116472957,
    'RECO': 116472959
}

In [ ]:
for dart_zone in dart_zones.keys():

    fig = pull_lmp_data(
        emtdb=emtdb,
        pnode_id=dart_zones[dart_zone],
        da_or_rt='DA',
        start_dt='2014-01-01',
        end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
        price_data_type='PRICE'
    ).rename(
        columns={
            'Price': 'DA'
        }
    ).reset_index().merge(pull_lmp_data(
        emtdb=emtdb,
        pnode_id=dart_zones[dart_zone],
        da_or_rt='RT',
        start_dt='2014-01-01',
        end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
        price_data_type='PRICE'
    ).reset_index().rename(
        columns={
            'Price': 'RT'
        }
    ), on=['Date', 'Hour']).assign(
        DART=lambda DF: DF.DA - DF.RT
    ).pipe(
        px.line,
        x='Date',
        y='DART',
        title=f'{dart_zone} DART spread (DA minus RT)',
        labels={
            'DART': 'DART ($/MWh)'
        }
    ).update_yaxes(range=[-500, 500])

    fig.show()

Ideally, for pricing an energy-pass through deal where we're paid on RT and incur costs on DA should have an covariance that is based on the DART spread. However, I think the expected value of the DART spread is close to 0 (checked below for PSEG), leading to a small covariance in practice. Hence we don't calculate the LFA on this deal.

In [ ]:
pull_lmp_data(
        emtdb=emtdb,
        pnode_id=dart_zones['PSEG'],
        da_or_rt='DA',
        start_dt='2014-01-01',
        end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
        price_data_type='PRICE'
    ).rename(
        columns={
            'Price': 'DA'
        }
    ).reset_index().merge(pull_lmp_data(
        emtdb=emtdb,
        pnode_id=dart_zones['PSEG'],
        da_or_rt='RT',
        start_dt='2014-01-01',
        end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
        price_data_type='PRICE'
    ).reset_index().rename(
        columns={
            'Price': 'RT'
        }
    ), on=['Date', 'Hour']).assign(
        DART=lambda DF: DF.DA - DF.RT
    ).merge(
        df_load_pivoted[['PSEG_BGS']].reset_index(),
        on=['Date', 'Hour']
    ).pipe(
        # lambda DF: DF['DART'].mean(), # Checking expectation of DART
        lambda DF: DF['DART'].cov(DF['PSEG_BGS']) # Checking covariance of DART with load
    )

#### Calculating DART hub-basis correlations

In [ ]:
for dart_zone in dart_zones.keys():

    fig = pull_lmp_data(
            emtdb=emtdb,
            pnode_id=dart_zones[dart_zone],
            da_or_rt='DA',
            start_dt='2014-01-01',
            end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
            price_data_type='PRICE'
        ).rename(
            columns={
                'Price': 'DA_zone'
            }
        ).reset_index().merge(pull_lmp_data(
            emtdb=emtdb,
            pnode_id=dart_zones[dart_zone],
            da_or_rt='RT',
            start_dt='2014-01-01',
            end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
            price_data_type='PRICE'
        ).reset_index().rename(
            columns={
                'Price': 'RT_zone'
            }
        ), on=['Date', 'Hour']).assign(
            DART_zone=lambda DF: DF.DA_zone - DF.RT_zone
        ).merge(pull_lmp_data(
            emtdb=emtdb,
            pnode_id=51288, # PJM-W
            da_or_rt='DA',
            start_dt='2014-01-01',
            end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
            price_data_type='PRICE'
        ).reset_index().rename(
            columns={
                'Price': 'DA_hub'
            }
        ), on=['Date', 'Hour']).merge(pull_lmp_data(
            emtdb=emtdb,
            pnode_id=51288, # PJM-W
            da_or_rt='RT',
            start_dt='2014-01-01',
            end_dt=pd.Timestamp.today().date() - pd.offsets.MonthEnd(),
            price_data_type='PRICE'
        ).reset_index().rename(
            columns={
                'Price': 'RT_hub'
            }
        ), on=['Date', 'Hour']).assign(
            DART_hub=lambda DF: DF.DA_hub - DF.RT_hub,
            DART_basis=lambda DF: DF.DART_zone - DF.DART_hub,
            Month=lambda DF: DF.Date.dt.month,
            Year=lambda DF: DF.Date.dt.year
        ).groupby(['Year', 'Month']).apply(
            lambda DF: DF.DART_basis.corr(DF.DART_hub)
        ).reset_index().pivot(
            index='Month',
            columns='Year',
            values=0
        ).pipe(
            px.imshow,
            title=f'{dart_zone} DART hub-basis correlations',
            # width=10,
            # height=10
        )

    fig.show()

### NJ REC Analysis

Array.from(document.querySelectorAll('a[href*=".xlsx"], a[href*=".xls"]'))
  .map(a => a.href)
  .forEach(url => console.log(url));

I used the above block of JS code to get the urls from https://njcleanenergy.com/renewable-energy/project-activity-reports/solar-activity-report-archive, copied them manually into an excel file and then downloaded them.

In [ ]:
import requests
from pathlib import Path

# Paste all the URLs you got from the browser here

urls = pd.read_excel(
    r'K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Output and Analysis\RECs\Solar Activity URLs.xlsx'
).iloc[:, 0].tolist()

download_dir = Path("nj_solar_reports")
download_dir.mkdir(exist_ok=True)

for idx, url in enumerate(urls, 1):
    try:
        filename = url.split('/')[-1].replace('+', '_')
        filepath = download_dir / filename
        
        print(f"[{idx}/{len(urls)}] Downloading {filename}...")
        response = requests.get(url, stream=True, timeout=30)
        
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)
        
        print(f"  ✓ Saved")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")

print(f"\nDone! Files in {download_dir.absolute()}")


### Working Capital Slides Automation - did not work in the end

In [2]:
# If this fails, try closing Excel in task manager - foreground and background processes

folder_path = r'K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Output and Analysis\Working Capital'

for file_name in os.listdir(folder_path):
    if 'Scenario' in file_name:
    
        file_path = os.path.join(folder_path, file_name)
        print(file_path)

        pythoncom.CoInitialize()
        try:
            # Open Excel
            excel = win32com.client.Dispatch("Excel.Application")
            excel.Visible = False  # Set to True if you want to see Excel
            
            # Open your workbook
            workbook = excel.Workbooks.Open(file_path)
            
            # Run the macro (use the macro name as it appears in VBA)
            excel.Application.Run("CreatePPT")
            
            # Save and close
            workbook.Save()
            workbook.Close()
            excel.Quit()
            
        finally:
            # Uninitialize COM
            pythoncom.CoUninitialize()

K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Output and Analysis\Working Capital\Pri Hemanth BGS-CIEP 2026-02 PJM WC Model V2.0 - ACE Scenario 1.xlsm
K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Output and Analysis\Working Capital\Pri Hemanth BGS-CIEP 2026-02 PJM WC Model V2.0 - ACE Scenario 2.xlsm


: 

### Exporting to Load Agg Excel file (while archiving if it already exists)

In [ ]:
load_agg_folder_path = r'K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Load'
load_agg_archived_folder_path = r'K:\Valuation\Wholesale for Retail\_Full Requirements\PJM\BGS FP & CIEP\2026-02\CIEP\Pri Hemanth\Load\Archived'
load_agg_file_name = 'Pri Hemanth BGS-CIEP 2026-02 Load Agg.xlsx'

load_agg_file_path = os.path.join(load_agg_folder_path, load_agg_file_name)
load_agg_archived_file_path = os.path.join(load_agg_archived_folder_path, load_agg_file_name)

if os.path.isfile(load_agg_file_path):
    os.makedirs(load_agg_archived_folder_path, exist_ok=True)
    
    if os.path.exists(load_agg_archived_file_path):
        print(f'Deleting old archived load agg...')
        os.remove(load_agg_archived_file_path)

    # os.rename(load_agg_file_path, load_agg_archived_file_path)
    print(f'Copying existing load agg to archived folder and removing the version in main folder...')
    shutil.copy2(load_agg_file_path, load_agg_archived_file_path)
    os.remove(load_agg_file_path)

with pd.ExcelWriter(load_agg_file_path, engine='openpyxl') as writer:
    print(f'Saving new load agg...')
    df_load_pivoted.to_excel(writer, sheet_name='Load'),
    df_tags_pivoted_unscaled.to_excel(writer, sheet_name='Raw Tags'), # All of these are unscaled
    df_deration_factors_pivoted.to_excel(writer, sheet_name='Deration Factors'),
    df_scaling_factors_pivoted.to_excel(writer, sheet_name='Scaling Factors')
    df_counts_pivoted.to_excel(writer, sheet_name='Counts')

## Auction

### Decrement formula

In [ ]:
def delta(gamma, tranche_target: int, regime: int):
    if regime == 1:
        
        if tranche_target >= 20:
            bins = [0.13, 0.28, 0.54, 0.775]
            values = [0.005, 0.0175, 0.03, 0.04, 0.05]
        
        elif 10 <= tranche_target <= 19:
            bins = [0.13, 0.21, 0.21, 0.28]
            values = [0.005, 0.0175, 0.03, 0.04, 0.05]

        elif 2 <= tranche_target <= 9:
            bins = [0.32, 0.55]
            values = [0.0175, 0.03, 0.05]
                        
        else: # 1 tranche
            bins = [0.2]
            values=[0.03, 0.05]
    
    elif regime == 2:
        
        if tranche_target >= 20:
            bins = [0.18, 0.34, 0.57, 0.75]
            values = [0.00375, 0.0125, 0.0225, 0.03, 0.0375]

        elif 10 <= tranche_target <= 19:
            bins = [0.18, 0.28, 0.38, 0.48]
            values = [0.00375, 0.0125, 0.0225, 0.03, 0.0375]

        elif 2 <= tranche_target <= 9:
            bins = [0.32, 0.55]
            values = [0.0125, 0.0225, 0.0375]

        else: # 1 tranche
            bins = [0.2]
            values=[0.0225, 0.0375] 
    
    else: # regime == 3
        
        if tranche_target >= 20:
            bins = [0.25, 0.75]
            values = [0.0025, 0.015, 0.025]

        elif 10 <= tranche_target <= 19:
            bins = [0.25, 0.6]
            values = [0.0025, 0.015, 0.025]

        elif 2 <= tranche_target <= 9:
            bins = [0.42]
            values=[0.015, 0.025] 

        else: # 1 tranche
            bins = [0.2]
            values=[0.015, 0.025] 
    
    indices = np.digitize(gamma, bins)
    return np.array(values)[indices]

In [ ]:
gamma = np.linspace(0, 1, 1000)
tranche_target = 24
regime = 2

px.line(
    x=gamma, 
    y=delta(gamma, tranche_target, regime),
    title=f'Regime {regime}, Tranche Target {tranche_target}',
    labels={
        'x': 'Gamma (Oversupply ratio)',
        'y': 'Delta (Decrement)'
    }
)